## Environmental impact assessment

## Mapping impact

Following field mission from the 29/06/2022 to the 02/07/2022

In [1]:
import xyzservices.providers as xyz
import geopandas as gpd
import fiona
import json

from os.path import exists

img_vector=["AbandonedPonds","Gates","MangroveArea","PondsClosetoMangrove","RestoredMangrove","SanjiangPark","Villages"]
geopkgList=[]
geojsonList=[]
data=[]

#open gpkg
for gpkgName in img_vector:
    for layername in fiona.listlayers(r'Vectors/Photos sites/'+gpkgName+'.gpkg'):
        geopkgList.append(gpd.read_file(r'Vectors/Photos sites/'+gpkgName+'.gpkg', layer=layername))

#convert to geojson (needed for on_click  popup method)
i=0
for i in range(len(geopkgList)):
    if exists('Vectors/Photos sites/geojson/'+img_vector[i]+'.geojson')==False:
        geojsonList.append(geopkgList[i].to_file('Vectors/Photos sites/geojson/'+img_vector[i]+'.geojson', driver='GeoJSON')) 

    with open('Vectors/Photos sites/geojson/'+img_vector[i]+'.geojson', 'r') as f:
        data.append(json.load(f))

print(len(data))

7


In [2]:
from ipywidgets import Label, Layout, Output, Image, HBox, Button, Box
from IPython.display import display, HTML
from ipyleaflet import Map, GeoData, basemaps, basemap_to_tiles, LayersControl, Marker, Popup, GeoJSON
import random
from PIL import Image as ImagePIL

photos_geojson_lst=[]

#import the GoogleSatellite xyz tileset
#tiles=xyz.GoogleSatellite
tiles={'url': 'http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}', 'max_zoom': 18, 'attribution': '(C) Google Maps', 'name': 'GoogleSatellite'}

#Coordinates of center of the bay (WGS)
center_bay=(19.96323,110.61723)

#geo_data = GeoData(geo_dataframe = geopkg)


#create map
m = Map(
    basemap=basemap_to_tiles(tiles),
    center=center_bay,
    zoom=12,
    scroll_wheel_zoom=True
    )

for i in range(len(data)):
    photos_geojson_lst.append(GeoJSON(
        data=data[i],
        point_style={
            'color': 'orange','opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1,'radius':5
        },
        hover_style={
            'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
        }
    ))

print(len(photos_geojson_lst))    
    
#define the handle_click (what happens when on_click is called - i.e. callback func)
def handle_click(feature, **kwargs):

#open image with PIL to get the image size (to define the box)    
    im = ImagePIL.open("IMG/"+feature['properties']['filename']+".jpg")
    ratio = im.size[0]/im.size[1]
    des_width_pic = 500
#call    
    box_layout = Layout(display='flex',
#                        flex_flow='column',
#                        align_items='stretch',
#                        border='solid',
                        width= str(des_width_pic)+'px',
                        height=str(des_width_pic/ratio)+'px')
    
    
    with open("IMG/"+feature['properties']['filename']+".jpg", "rb") as f:
        image = f.read()
    items = Image(
        value=image,
        format ='jpg'
        )
    items.layout.object_fit="cover"
    box = Box(children=[items], layout=box_layout)
    
    #define the popup that will appear
    popupClick = Popup(
        location=(feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0]),
        max_width=1000,
        child=box,
        close_button=True,

    )

    m.add_layer(popupClick)

for i in range(len(photos_geojson_lst)):
    #attach on click action to geojson
    photos_geojson_lst[i].on_click(handle_click)

    #add geojson
    m.add_layer(photos_geojson_lst[i])

#add layers control
m.add_control(LayersControl())

m




7


Map(center=[19.96323, 110.61723], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [22]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p xyzservices,geopandas,fiona,json,ipywidgets,IPython,ipyleaflet,PIL,watermark


# date
print (" ")

%watermark -u -n -t -z 


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.4.0

xyzservices: 2022.6.0
geopandas  : 0.10.2
fiona      : 1.8.21
json       : 2.0.9
ipywidgets : 7.7.1
IPython    : 8.4.0
ipyleaflet : 0.17.0
PIL        : 9.1.1
watermark  : 2.3.1

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

 
Last updated: Sat Jul 16 2022 04:35:32China Standard Time

